In [35]:
import numpy as np
import pandas as pd
import math 

from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import PolynomialFeatures, normalize
from doubleml import DoubleMLData, DoubleMLPLR

import warnings
warnings.filterwarnings('ignore')

import os
os.cpu_count()


8

# DML with 4 covariate model plus region and district fixed effects

#### By Shahin Karami

# Importing Data 

In [36]:
#importing data.
#First converted the data to a CSV via Stata so i can inport

df = pd.read_excel('sle_merged_altered.xlsx')
df.head(10)
dfcopy = df.copy()

collist = list(dfcopy.columns)

# Y variable 

In [37]:
# dependent var that we are looking into 

y = dfcopy[["student_knowledge"]] # are we sure we want to use "student_knowledge" and not "student_proficiency"?
#y

# 4 Covariates + District/Region Effects Nuisence Parameters (X vars)

In [38]:
Regions = pd.get_dummies(df["ADM1_NAME"])
Districts = pd.get_dummies(df["ADM2_NAME"])
dfcopy = pd.concat([dfcopy, Regions], axis=1)
dfcopy = pd.concat([dfcopy, Districts], axis=1)

In [39]:
# Nuisense Parameters... These are all from the "Variable name" column in the GECD codebook, except for light_GDP

X4_list = ["students_enrolled","infrastructure", "light_GDP","ecd_student_proficiency","Eastern", "Northern", "Southern", 
          "Western Area", "Bo", "Bombali", "Bonthe", "Kailahun", "Kambia", "Kenema", "Koinadugu", "Kono", 
          "Moyamba","Port Loko", "Pujehun","Tonkolili", "Western Area Rur","Western Area Urb"]

X4 = dfcopy[X4_list]

X4.shape
X4.columns


Index(['students_enrolled', 'infrastructure', 'light_GDP',
       'ecd_student_proficiency', 'Eastern', 'Northern', 'Southern',
       'Western Area', 'Bo', 'Bombali', 'Bonthe', 'Kailahun', 'Kambia',
       'Kenema', 'Koinadugu', 'Kono', 'Moyamba', 'Port Loko', 'Pujehun',
       'Tonkolili', 'Western Area Rur', 'Western Area Urb'],
      dtype='object')

# Policy of Intrest (D Vars) Quasi-Arithmetic Mean with Weighted Aggregation

In [40]:
BE_alt = dfcopy[["bureaucratic_efficiency_alt"]]
IDM_alt = dfcopy[["impartial_decision_making_alt"]]
MA_alt = dfcopy[["mandates_accountability_alt"]]
QOB_alt = dfcopy[["quality_bureaucracy_alt"]]
NLG_alt = dfcopy[["national_learning_goals_alt"]]

# Policy of Intrest (D Vars) Quasi-Arithmetic Mean

In [41]:
BE_quas = dfcopy[["bureaucratic_efficiency_quas"]]
IDM_quas = dfcopy[["impartial_decision_making_quas"]]
MA_quas = dfcopy[["mandates_accountability_quas"]]
QOB_quas = dfcopy[["quality_bureaucracy_quas"]]
NLG_quas = dfcopy[["national_learning_goals_quas"]]

# Policy of Intrest (D Vars) Arithmetic Mean with Weighted Aggregation

In [42]:
BE_cw = dfcopy[["bureaucratic_efficiency_cw"]]
IDM_cw = dfcopy[["impartial_decision_making_cw"]]
MA_cw = dfcopy[["mandates_accountability_cw"]]
QOB_cw = dfcopy[["quality_bureaucracy_cw"]]
NLG_cw = dfcopy[["national_learning_goals_cw"]]

# Policy of Intrest (D Vars) Arithmetic Mean

In [43]:
BE = dfcopy[["bureaucratic_efficiency"]]
IDM = dfcopy[["impartial_decision_making"]]
MA = dfcopy[["mandates_accountability"]]
QOB = dfcopy[["quality_bureaucracy"]]
NLG = dfcopy[["national_learning_goals"]]

# Combining y, D, X

In this chunk of code you can define which index you would like to choose. Namely you have the choice between the four indexes created above.

1. Arithmetic Mean: BE, IDM, MA, QOB, NLG 

2. Arithmetic Mean with Weighted Aggregation: BE_cw, IDM_cw, MA_cw, QOB_cw, NLG_cw

3. Quasi-Arithmetic: BE_quas, IDM_quas, MA_quas, QOB_quas, NLG_quas

4. Quasi-Arithmetic with Weighted Aggregation: BE_alt, IDM_alt, MA_alt, QOB_alt, NLG_alt

Once you define the index you want here you wont have to alter any other code chunk. 

In [44]:
# Combining all my columns into a single new dataframe
combined_df4 = pd.DataFrame()
combined_df4["Student Knowledge"] = y

# Make sure define the correct index you want
combined_df4["Bureaucratic Efficiency"] = BE
combined_df4["Impartial Decision Making"] = IDM
combined_df4["Mandates & Accountability"] = MA
combined_df4["Quality of Bureaucracy"] = QOB
combined_df4["National Learning Goals"] = NLG

combined_df4 = pd.concat([combined_df4, X4], axis=1)
combined_df4.describe()

,Student Knowledge,Bureaucratic Efficiency,Impartial Decision Making,Mandates & Accountability,Quality of Bureaucracy,National Learning Goals,students_enrolled,infrastructure,light_GDP,ecd_student_proficiency,...,Kambia,Kenema,Koinadugu,Kono,Moyamba,Port Loko,Pujehun,Tonkolili,Western Area Rur,Western Area Urb
count,298.000000,301.000000,301.000000,301.000000,301.000000,301.000000,299.000000,297.000000,301.000000,300.000000,...,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000
mean,36.815271,3.641350,3.727466,3.691594,3.927825,3.218516,315.969900,1.789883,0.849009,28.755556,...,0.072848,0.096026,0.059603,0.076159,0.069536,0.109272,0.043046,0.072848,0.062914,0.056291
std,16.549401,0.247292,0.338749,0.483390,0.246761,0.424640,191.715592,0.956309,2.421012,39.364228,...,0.260318,0.295117,0.237142,0.265693,0.254786,0.312497,0.203298,0.260318,0.243211,0.230866
min,9.001401,3.170291,2.606602,2.616667,3.224265,2.400000,40.000000,0.142857,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.745766,3.430261,3.655909,3.450397,3.804248,3.111953,200.000000,1.285714,0.009674,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,34.291769,3.647455,3.781818,3.904321,3.931871,3.188333,289.000000,1.714286,0.020960,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,48.952332,3.819554,3.972222,4.088889,4.055247,3.588384,385.500000,2.571429,0.253130,66.666667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,92.380392,3.995975,4.078520,4.256944,4.535539,3.792689,2006.000000,4.857143,26.278387,100.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Cleaning up NA's and Creating Higher Order and Interaction Variables

In [47]:
# Cleaning dataframe to drop NaN, Then split Nuisense param so I can run Polynomial feature
combined_clean4 = combined_df4.dropna()
combined_clean4 = combined_clean4.reset_index(drop=True)
combined_clean4.describe()


X4_update = combined_clean4.iloc[:,6:] # only selecting the covariates and not y or D

X4_update = normalize(X4_update) # we only normalize the covariates

X4_update = pd.DataFrame(X4_update, columns=X4.columns)

poly = PolynomialFeatures(degree=3) # default is to include_bias = True in other words there will be a column of 1s added
X4_poly = poly.fit_transform(X4_update)
X4_poly_df = pd.DataFrame(X4_poly, columns=poly.get_feature_names(X4.columns))

# pulling all the column names into a list for the DoubleMLData
X4_col = list(X4_poly_df.columns)  

X4_poly_df.shape
X4_poly_df # all the covariates

,1,students_enrolled,infrastructure,light_GDP,ecd_student_proficiency,Eastern,Northern,Southern,Western Area,Bo,...,Tonkolili^3,Tonkolili^2 Western Area Rur,Tonkolili^2 Western Area Urb,Tonkolili Western Area Rur^2,Tonkolili Western Area Rur Western Area Urb,Tonkolili Western Area Urb^2,Western Area Rur^3,Western Area Rur^2 Western Area Urb,Western Area Rur Western Area Urb^2,Western Area Urb^3
0,1.0,0.999982,0.002223,0.000048,0.000000,0.003891,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
1,1.0,0.999987,0.004064,0.000362,0.000000,0.002188,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
2,1.0,0.999932,0.004362,0.000034,0.000000,0.007633,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
3,1.0,0.996788,0.009198,0.000027,0.079489,0.002385,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
4,1.0,0.999898,0.012682,0.000065,0.000000,0.004672,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,1.0,0.724061,0.010836,0.000067,0.689582,0.000000,0.0,0.006896,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
291,1.0,0.999774,0.012242,0.010997,0.000000,0.000000,0.0,0.000000,0.009522,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.632518e-07,0.0,0.0,0.000000e+00
292,1.0,0.846304,0.011406,0.016552,0.532267,0.000000,0.0,0.000000,0.005323,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1.507955e-07
293,1.0,0.907864,0.010758,0.024451,0.418371,0.000000,0.0,0.000000,0.004184,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,7.322907e-08


# Recombining y, D, X (Now without NA's)

In [48]:
# Combining all my columns into a single new dataframe
combined_clean_poly4 = pd.DataFrame()
combined_clean_poly4["Student Knowledge"] = combined_clean4["Student Knowledge"]
combined_clean_poly4["Bureaucratic Efficiency"] = combined_clean4["Bureaucratic Efficiency"]
combined_clean_poly4["Impartial Decision Making"] = combined_clean4["Impartial Decision Making"]
combined_clean_poly4["Mandates & Accountability"] = combined_clean4["Mandates & Accountability"]
combined_clean_poly4["Quality of Bureaucracy"] = combined_clean4["Quality of Bureaucracy"]
combined_clean_poly4["National Learning Goals"] = combined_clean4["National Learning Goals"]
combined_clean_poly4 = pd.concat([combined_clean_poly4, X4_poly_df], axis=1)

combined_clean_poly4.describe()
combined_clean_poly4

,Student Knowledge,Bureaucratic Efficiency,Impartial Decision Making,Mandates & Accountability,Quality of Bureaucracy,National Learning Goals,1,students_enrolled,infrastructure,light_GDP,...,Tonkolili^3,Tonkolili^2 Western Area Rur,Tonkolili^2 Western Area Urb,Tonkolili Western Area Rur^2,Tonkolili Western Area Rur Western Area Urb,Tonkolili Western Area Urb^2,Western Area Rur^3,Western Area Rur^2 Western Area Urb,Western Area Rur Western Area Urb^2,Western Area Urb^3
0,28.235294,3.381969,2.606602,3.904321,3.224265,3.792689,1.0,0.999982,0.002223,0.000048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
1,30.737745,3.381969,2.606602,3.904321,3.224265,3.792689,1.0,0.999987,0.004064,0.000362,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
2,32.664760,3.381969,2.606602,3.904321,3.224265,3.792689,1.0,0.999932,0.004362,0.000034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
3,31.884532,3.381969,2.606602,3.904321,3.224265,3.792689,1.0,0.996788,0.009198,0.000027,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
4,76.840959,3.381969,2.606602,3.904321,3.224265,3.792689,1.0,0.999898,0.012682,0.000065,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,26.045752,3.589288,3.601010,3.625000,3.942810,3.188333,1.0,0.724061,0.010836,0.000067,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00
291,36.374592,3.793360,3.733333,4.088889,3.804248,3.546970,1.0,0.999774,0.012242,0.010997,...,0.0,0.0,0.0,0.0,0.0,0.0,8.632518e-07,0.0,0.0,0.000000e+00
292,58.315178,3.633115,4.068485,3.620437,3.931871,2.911667,1.0,0.846304,0.011406,0.016552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,1.507955e-07
293,66.573393,3.633115,4.068485,3.620437,3.931871,2.911667,1.0,0.907864,0.010758,0.024451,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,7.322907e-08


# Creating the DoubleMLData Objects for Each Policy Var (5 total)

To use the DoubleML package the user must first create a DoubleMLData object. In this Object you define what your y,d,x as well as the data you are going to be calling your model parameters from. DoubleMLData can also include a `z_cols` argument should the user want to include a intrumental variable to the analysis.

More information on the DoubleML package can be found for both the python and R APIs here: https://docs.doubleml.org/stable/index.html

In [49]:
# Bureaucratic Efficiency
dmlPLR_data_BE_4 = DoubleMLData(
    combined_clean_poly4,
    y_col='Student Knowledge',
    d_cols='Bureaucratic Efficiency',
    x_cols= X4_col
)

# Impartial Desicion Making
dmlPLR_data_IDM_4 = DoubleMLData(
    combined_clean_poly4,
    y_col='Student Knowledge',
    d_cols='Impartial Decision Making',
    x_cols=X4_col
)

# Mandates & Accountability
dmlPLR_data_MA_4 = DoubleMLData(
    combined_clean_poly4,
    y_col='Student Knowledge',
    d_cols='Mandates & Accountability',
    x_cols=X4_col
)

# Quality of Bureaucracy
dmlPLR_data_QOB_4 = DoubleMLData(
    combined_clean_poly4,
    y_col='Student Knowledge',
    d_cols='Quality of Bureaucracy',
    x_cols=X4_col
)

# National Learning Goals
dmlPLR_data_NLG_4 = DoubleMLData(
    combined_clean_poly4,
    y_col='Student Knowledge',
    d_cols='National Learning Goals',
    x_cols=X4_col
)

# Partial Linear Regrssion DoubleML using 

Note that any Machine Learing model can be used in the DoubleML Package. We opt to use a linear model fitted by minimizing a regularized empirical loss with Stochastic Gradient Descent (SGD) from `sklearn`. 

Refer to the documentation for sklearn.linear_model.SGDRegressor for more information 

In [51]:
# Run Partial Linear Regrssion DoubleML with the various DoubleML datasets

# Bureaucratic Efficiency  
dml_plrSGDR_BE_4 = DoubleMLPLR(
    obj_dml_data=dmlPLR_data_BE_4,
    ml_l=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    ml_m=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    n_folds=2
)

BE_4 = dml_plrSGDR_BE_4.fit().fit(n_jobs_cv = os.cpu_count()).bootstrap(method='normal', n_rep_boot=500000)

# Impartial Desicion Making
dml_plrSGDR_IDM_4 = DoubleMLPLR(
    obj_dml_data=dmlPLR_data_IDM_4,
    ml_l=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    ml_m=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    n_folds=2
)

IDM_4 = dml_plrSGDR_IDM_4.fit(n_jobs_cv = os.cpu_count()).bootstrap(method='normal', n_rep_boot=500000)

# Mandates & Accountability
dml_plrSGDR_MA_4 = DoubleMLPLR(
    obj_dml_data=dmlPLR_data_MA_4,
    ml_l=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    ml_m=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    n_folds=2
)

MA_4 = dml_plrSGDR_MA_4.fit(n_jobs_cv = os.cpu_count()).bootstrap(method='normal', n_rep_boot=500000)

# Quality of Bureaucracy
dml_plrSGDR_QOB_4 = DoubleMLPLR(
    obj_dml_data=dmlPLR_data_QOB_4,
    ml_l=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    ml_m=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    n_folds=2
)

QOB_4 = dml_plrSGDR_QOB_4.fit(n_jobs_cv = os.cpu_count()).bootstrap(method='normal', n_rep_boot=500000)

# National Learning Goals
dml_plrSGDR_NLG_4 = DoubleMLPLR(
    obj_dml_data=dmlPLR_data_NLG_4,
    ml_l=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    ml_m=SGDRegressor(learning_rate = "optimal",alpha = 0.0001),
    n_folds=2
)

NLG_4 = dml_plrSGDR_NLG_4.fit(n_jobs_cv = os.cpu_count()).bootstrap(method='normal', n_rep_boot=500000)

dml_plr_SGDR = pd.concat([BE_4.summary, IDM_4.summary, MA_4.summary, QOB_4.summary, NLG_4.summary])
dml_plr_SGDR

# In order to save results from all 4 indicies I alter the name of the dml_plr_SGDR 
# to include the indicies within the variable name (see below for examples)
# due to time constraints I was not able to automate this process

,coef,std err,t,P>|t|,2.5 %,97.5 %
Bureaucratic Efficiency,0.187555,0.042910,4.370927,1.237199e-05,0.103454,0.271657
Impartial Decision Making,0.787223,0.020482,38.435097,0.000000e+00,0.747079,0.827366
Mandates & Accountability,-0.569549,0.021517,-26.469879,2.154642e-154,-0.611721,-0.527376
Quality of Bureaucracy,0.433246,0.016921,25.604539,1.357933e-144,0.400082,0.466410
National Learning Goals,-0.310562,0.094967,-3.270223,1.074628e-03,-0.496693,-0.124431


# SGDRegressor with Quasi-Arithmetic Mean with Weighted Aggregeation


In [17]:
dml_plr_SGDR_alt

,coef,std err,t,P>|t|,2.5 %,97.5 %
Bureaucratic Efficiency,-0.134032,0.020754,-6.458247,1.059228e-10,-0.174709,-0.093356
Impartial Decision Making,0.747803,0.054566,13.704486,9.544130e-43,0.640855,0.854751
Mandates & Accountability,-0.381462,0.024671,-15.462179,6.244942e-54,-0.429816,-0.333109
Quality of Bureaucracy,-0.868319,0.053058,-16.365444,3.375572e-60,-0.972311,-0.764327
National Learning Goals,-0.119921,0.131606,-0.911217,3.621810e-01,-0.377863,0.138021


# SGDRegressor with Quasi-Arithmetic Mean


In [32]:
dml_plr_SGDR_quas

,coef,std err,t,P>|t|,2.5 %,97.5 %
Bureaucratic Efficiency,-0.235000,0.018507,-12.698112,6.057346e-37,-0.271273,-0.198728
Impartial Decision Making,0.419727,0.017994,23.326213,2.403426e-120,0.384460,0.454994
Mandates & Accountability,0.144163,0.008276,17.420291,5.788155e-68,0.127944,0.160383
Quality of Bureaucracy,0.009618,0.001775,5.417828,6.032732e-08,0.006139,0.013098
National Learning Goals,0.509738,0.072413,7.039285,1.932283e-12,0.367811,0.651666


# SGDRegressor with Arithmetic Mean and Weighted aggregeation


In [68]:
 dml_plr_SGDR_cw

,coef,std err,t,P>|t|,2.5 %,97.5 %
Bureaucratic Efficiency,0.964283,0.051973,18.553643,7.620809e-77,0.862418,1.066148
Impartial Decision Making,0.525790,0.052264,10.060185,8.284277e-24,0.423353,0.628226
Mandates & Accountability,-0.948383,0.035140,-26.988947,1.992524e-160,-1.017255,-0.879510
Quality of Bureaucracy,-0.254879,0.030971,-8.229476,1.880347e-16,-0.315582,-0.194176
National Learning Goals,1.267730,0.151766,8.353191,6.644333e-17,0.970274,1.565186


# SGDRegressor with Arithmetic Mean 

In [51]:
dml_plr_SGDR_raw

,coef,std err,t,P>|t|,2.5 %,97.5 %
Bureaucratic Efficiency,0.580960,0.020923,27.766795,1.092549e-169,0.539952,0.621968
Impartial Decision Making,1.209707,0.100379,12.051421,1.906345e-33,1.012968,1.406445
Mandates & Accountability,0.536688,0.020896,25.683820,1.772450e-145,0.495732,0.577643
Quality of Bureaucracy,-0.223298,0.017752,-12.578855,2.759999e-36,-0.258090,-0.188505
National Learning Goals,0.465358,0.075839,6.136127,8.455764e-10,0.316716,0.614000
